In [5]:
import datetime as dt
import pymysql, talib, sys, os, tempfile
import pandas as pd , numpy as np

#############################################
class HKta:
    conn = None
    symbol=None
    
    raw=None
    daily=None
    weekly=None
    monthly=None
    yearly=None
    ####################
    def __init__(self,symbol):
        self.symbol = symbol
        self.populate()
    ####################
    def populate(self):
        if self.conn is None:
            self.conn = pymysql.connect(host='tptcn.ddns.net', port=53306, db='p_master',
                                   user=os.environ['MYSQL_READUSER'], password=os.environ['MYSQL_READPASSWORD'],
                                   cursorclass=pymysql.cursors.DictCursor)

        tmpl = "SELECT  * FROM `consolidated_daily` WHERE `symbol`='%s' "
        sql = tmpl % (self.symbol)
        self.raw= pd.read_sql(sql, self.conn, index_col=['Date'] )[['Open', 'High', 'Low', 'Close', 'Volume']]
        ### drop NA 
        self.daily = self.raw.fillna(0)
        self.daily['Volume'] = self.daily['Volume'].astype('int')
        self.daily = self.daily[self.daily['Volume'] > 0]
        self.daily = self.daily.reset_index()
        self.daily['Date'] = pd.to_datetime(self.daily['Date'])
        self.daily=self.daily.set_index('Date')

        #Close should be official; so adjust High/Low if necessary
        self.daily['High'] = np.where(self.daily['Close'] > self.daily['High'], self.daily['Close'], self.daily['High'])
        self.daily['Low']  = np.where(self.daily['Close'] < self.daily['Low'],  self.daily['Close'], self.daily['Low'])
        #Open is calculated; so, adjust according to the High/Low if necessary
        self.daily['Open'] = np.where(self.daily['Open']  > self.daily['High'], self.daily['High'],  self.daily['Open'])
        self.daily['Open'] = np.where(self.daily['Open']  < self.daily['Low'],  self.daily['Low'],   self.daily['Open'])
         
        ### roll up 
        tmpdf = self.daily
        tmpdf = tmpdf.reset_index()

        tmpdf['yyyymm'] = tmpdf['Date'].dt.strftime("%Y%m")
        tmpdf['Month_Number'] = tmpdf['Date'].dt.month
        tmpdf['Year'] = tmpdf['Date'].dt.year

        # 'daysoffset' will container the weekday, as integers
        tmpdf['daysoffset'] = tmpdf['Date'].apply(lambda x: x.weekday())
        # We apply, row by row (axis=1) a timedelta operation
        tmpdf['WeekStart'] = tmpdf.apply(lambda x: x['Date'] - dt.timedelta(days=x['daysoffset']), axis=1)

        ### weekly
        self.weekly = tmpdf.groupby(['WeekStart']).agg({'Open':'first', 'High':'max', 'Low':'min', 'Close':'last', 'Volume':'sum'})
        self.weekly['Count'] =tmpdf.groupby(['WeekStart']).size()
        self.weekly['Volume'] = self.weekly['Volume'].astype('int')
        self.weekly['AvgDVol'] = (self.weekly['Volume'] / self.weekly['Count']).astype('int')
        self.weekly = self.weekly.reset_index().set_index('WeekStart')[['Open', 'High', 'Low', 'Close', 'Volume', 'AvgDVol', 'Count']]

        ### monthly
        self.monthly = tmpdf.groupby(['Year','Month_Number']).agg({'Open':'first', 'High':'max', 'Low':'min', 'Close':'last', 'Volume':'sum', 'yyyymm':'first'})
        self.monthly['Count'] =tmpdf.groupby(['Year','Month_Number']).size()
        self.monthly['Volume'] = self.monthly['Volume'].astype('int')
        self.monthly['AvgDVol'] = (self.monthly['Volume'] / self.monthly['Count']).astype('int')
        self.monthly = self.monthly.reset_index().set_index('yyyymm')[['Open', 'High', 'Low', 'Close', 'Volume', 'AvgDVol', 'Count']]

        ### yearly
        self.yearly = tmpdf.groupby(['Year']).agg({'Open':'first', 'High':'max', 'Low':'min', 'Close':'last', 'Volume':'sum'})
        self.yearly['Count'] =tmpdf.groupby(['Year']).size()
        self.yearly['Volume'] = self.yearly['Volume'].astype('int')
        self.yearly['AvgDVol'] = (self.yearly['Volume'] / self.yearly['Count']).astype('int')
        self.yearly = self.yearly[['Open', 'High', 'Low', 'Close', 'Volume', 'AvgDVol', 'Count']]


    ####################
    def __del__(self):
        if self.conn is not None:
            self.conn.close()
            self.conn = None


In [6]:
data = HKta('2388.HK')
df = data.daily
### daily done, calc TA
df['WCLPRICE'] = talib.WCLPRICE(df['High'], df['Low'], df['Close'])
print df

             Open   High    Low  Close     Volume  WCLPRICE
Date                                                       
2002-07-25   8.50   8.55   8.00   8.05  349715000    8.1625
2002-07-26   8.00   8.30   8.00   8.30  108773800    8.2250
2002-07-29   8.30   8.45   8.30   8.45   43057500    8.4125
2002-07-30   8.55   8.60   8.40   8.40   47204500    8.4500
2002-07-31   8.40   8.50   8.35   8.45   16484000    8.4375
2002-08-01   8.50   8.50   8.40   8.45   14545500    8.4500
2002-08-02   8.35   8.40   8.30   8.35   13508500    8.3500
2002-08-05   8.25   8.35   8.25   8.30   10141000    8.3000
2002-08-06   8.20   8.25   8.15   8.20   13995000    8.2000
2002-08-07   8.30   8.35   8.25   8.30    8749000    8.3000
2002-08-08   8.25   8.30   8.20   8.25    8245000    8.2500
2002-08-09   8.30   8.35   8.25   8.30    7818331    8.3000
2002-08-12   8.25   8.30   8.20   8.25    6566500    8.2500
2002-08-13   8.20   8.30   8.15   8.30    9060000    8.2625
2002-08-14   8.20   8.25   8.15   8.25  

In [8]:
for y in range(2008,2020):
    yyyyStart="%d-01-01"%(y)
    yyyyEnd  ="%d-12-31"%(y)
    df = data.daily.loc[yyyyStart:yyyyEnd]
    #print df
    

In [10]:
for y in range(2008,2020):
    yyyyStart="%d-01-01"%(y)
    yyyyEnd  ="%d-12-31"%(y)
    df = data.daily.loc[yyyyStart:yyyyEnd]
    print "year=%4d, mean=%.3f, stddev=%.3f, open=%.3f, high=%.3f, low=%.3f, rng=%.3f, close=%.3f, count=%d" % (
        y, df.WCLPRICE.mean(), df.WCLPRICE.std()
        , df.iloc[0]['WCLPRICE']  #Open
        , df['WCLPRICE'].max() #High
        , df['WCLPRICE'].min() #Low
        , df['WCLPRICE'].max() - df['WCLPRICE'].min() #Low
        , df.iloc[-1]['WCLPRICE'] #Close
        , len(df.index)  # count
    )
    #print

year=2008, mean=16.660, stddev=4.547, open=21.475, high=23.575, low=7.684, rng=15.891, close=8.738, count=240
year=2009, mean=13.436, stddev=3.996, open=8.995, high=19.715, low=6.393, rng=13.322, close=17.695, count=249
year=2010, mean=20.622, stddev=3.676, open=17.515, high=28.950, low=16.080, rng=12.870, close=26.375, count=249
year=2011, mean=21.948, stddev=3.148, open=26.688, high=28.025, low=14.685, rng=13.340, close=18.440, count=246
year=2012, mean=22.931, stddev=1.469, open=18.715, high=24.875, low=18.355, rng=6.520, close=24.062, count=245
year=2013, mean=25.456, stddev=1.011, open=24.487, high=27.825, low=23.175, rng=4.650, close=24.863, count=244
year=2014, mean=24.281, stddev=1.413, open=24.775, high=27.625, low=21.775, rng=5.850, close=25.925, count=236
year=2015, mean=27.652, stddev=3.055, open=25.950, high=33.400, low=22.488, rng=10.913, close=23.700, count=233
year=2016, mean=24.508, stddev=2.830, open=23.225, high=29.400, low=18.995, rng=10.405, close=27.675, count=245

In [11]:
t =dt.datetime.now()-dt.timedelta(days=365)
yStart= dt.datetime(t.year, t.month, t.day, tzinfo=t.tzinfo)
yEnd= dt.datetime(dt.datetime.now().year, dt.datetime.now().month, dt.datetime.now().day, tzinfo=dt.datetime.now().tzinfo)

df = data.daily.loc[yStart:yEnd]
print "year=%4d, mean=%.3f, stddev=%.3f, open=%.3f, high=%.3f, low=%.3f, rng=%.3f, close=%.3f, count=%d" % (
    y, df.WCLPRICE.mean(), df.WCLPRICE.std()
    , df.iloc[0]['WCLPRICE']  #Open
    , df['WCLPRICE'].max() #High
    , df['WCLPRICE'].min() #Low
    , df['WCLPRICE'].max() - df['WCLPRICE'].min() #Low
    , df.iloc[-1]['WCLPRICE'] #Close
    , len(df.index)  # count
)


year=2019, mean=35.657, stddev=4.079, open=37.538, high=41.100, low=27.963, rng=13.137, close=30.188, count=243


### Assumption 2388:
### assume 2019 range 27 - 37 ; mean at 32 ; stddev at 3

In [21]:
from scipy import stats
n = stats.norm( 30.5, 3.0 )

for q in np.linspace(10.0,90.0, 9):
    print "%2.1f  %.1f " %(100 - q, (n.ppf(q/100)) )
print
for p in np.linspace(37.5,26.5, 12):
    print "%2.2f  %.2f " %(p, 100-(n.cdf(p) * 100) )


90.0  26.7 
80.0  28.0 
70.0  28.9 
60.0  29.7 
50.0  30.5 
40.0  31.3 
30.0  32.1 
20.0  33.0 
10.0  34.3 

37.50  0.98 
36.50  2.28 
35.50  4.78 
34.50  9.12 
33.50  15.87 
32.50  25.25 
31.50  36.94 
30.50  50.00 
29.50  63.06 
28.50  74.75 
27.50  84.13 
26.50  90.88 
